In [1]:
from torch import nn, optim, cuda
import torch.nn.functional as F
from torchvision.transforms import transforms
from torchvision import datasets, models
from torch.utils.data import DataLoader
from pytorch_lightning import LightningModule, Trainer

In [2]:
def load_dataloader(name, transform, bs):
  if name == 'cifar10':
    train_ds = datasets.CIFAR10('./data', train=True, download=True, transform=transform)
    val_ds = datasets.CIFAR10('./data', train=False, download=True, transform=transform)
    # num_workers is the number of threads for loading data.  Pytorch lightning complains if 
    # this is not larger than 1.  It is ok for it to be 1 in general for small problems.
    train_dl = DataLoader(train_ds, bs, shuffle=True, num_workers=2)
    val_dl = DataLoader(val_ds, bs, shuffle=False, num_workers=2)
    
    return train_dl, val_dl    

In [3]:
class Model(LightningModule):
  def __init__(self, model='resnet18', freeze=False):
    super().__init__()
    self.model = self._build_model(model, freeze)

  def _build_model(self, name, freeze):
    if name=='resnet18':
      model = models.resnet18(pretrained=True)

      if freeze==True:
        for params in model.parameters():
          params.requires_grad = False
        model.fc = nn.Linear(512, 10)
        return model
      else:
        model.fc = nn.Linear(512, 10)
        return model

  def forward(x):
    out = self.model(x)
    return out

  def training_step(self, batch, batch_idx):
    x, y = batch
    out = self.model(x)
    loss = F.cross_entropy(out, y)
    self.log("train_loss", loss, prog_bar=True)

    return loss

  def validation_step(self, batch, batch_idx):
    x, y = batch
    out = self.model(x)
    loss = F.cross_entropy(out, y)
    self.log("val_loss", loss, prog_bar=True)

    return loss

  def configure_optimizers(self):
    return optim.Adam(self.model.parameters())

In [4]:
trainloader, valloader = load_dataloader('cifar10', transforms.ToTensor(), 64)
model = Model('resnet18', freeze=True)

Files already downloaded and verified


Files already downloaded and verified


/work/rcpaffenroth/opt/anaconda3/lib/python3.8/site-packages/torch/utils/data/dataloader.py:474: UserWarning: This DataLoader will create 2 worker processes in total. Our suggested max number of worker in current system is 1, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [5]:
if cuda.is_available():
    print('CUDA available')
    print('current device', cuda.current_device())
    print('device count', cuda.device_count())
    print('device name', cuda.get_device_name(0))
    print('arch list', cuda.get_arch_list())
    number_of_gpus=1
else:
    print('CUDA *not* available')
    number_of_gpus=0

CUDA available
current device 0
device count 1
device name Tesla P100-SXM2-16GB
arch list ['sm_37', 'sm_50', 'sm_60', 'sm_61', 'sm_70', 'sm_75', 'sm_80', 'sm_86', 'compute_37']


In [6]:
trainer = Trainer(max_epochs=2,
                  fast_dev_run=True, # This runs just one batch to make sure things are running.  REMOVE FOR NORMAL RUNS!
                  gpus=number_of_gpus)

GPU available: True, used: True


TPU available: False, using: 0 TPU cores


Running in fast_dev_run mode: will run a full train, val and test loop using 1 batch(es).


In [7]:
trainer.fit(model, trainloader, valloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Set SLURM handle signals.



  | Name  | Type   | Params
---------------------------------
0 | model | ResNet | 11.2 M
---------------------------------
5.1 K     Trainable params
11.2 M    Non-trainable params
11.2 M    Total params
44.727    Total estimated model params size (MB)


/work/rcpaffenroth/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)
/work/rcpaffenroth/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/utilities/distributed.py:69: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]